<a href="https://colab.research.google.com/github/Jnfagundes/EstudoUNOPAR/blob/main/BuscaNoticia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datetime import datetime #Importação da função datatime da biblioteca datatime

import requests # Importação da biblioteca
from bs4 import BeautifulSoup
import pandas as pd

#tilizar a propriedade text da biblioteca requests para capturar em formato de string. 
#Em seguida, vamos transformar essa string em formato html, para que possamos localizar as tags de nosso interesse. #
#Na linha 2, registramos o horário da extração. 
#Na linha 5, procuramos todas as tags div com o atributo que nos foi indicado. Essa linha retornará uma lista com cada notícia. 
#Na linha 6 imprimimos quantas notícias foram encontradas e na linha 7 imprimimos o conteúdo da primeira notícia.
texto_string = requests.get('https://globoesporte.globo.com/').text
hora_extracao = datetime.now().strftime("%d-%m-%Y %H:%M:%S")

bsp_texto = BeautifulSoup(texto_string, 'html.parser')
lista_noticias = bsp_texto.find_all('div', attrs={'class':'feed-post-body'})
print("Quantidade de manchetes = ", len(lista_noticias))
lista_noticias[0].contents # contents transforma cada início e final da div em um elemento da lista.


#Dentro dessa estrutura, procurando pelas tags corretas, vamos encontrar todas as informações que foram solicitadas. 
lista_noticias[0].contents[1].text.replace('"',"")

#Para extração do link para notícia, como ele se encontra também na posição 1 da lista, vamos utilizar o método find('a') para localizá-lo e extrair
lista_noticias[0].find('a').get('href')

#Para a descrição, como ela pode estar na terceira posição ou em outra tag, vamos ter que testar em ambas e caso não esteja, então retornar None (nulo).
descricao = lista_noticias[0].contents[2].text
if not descricao:
    descricao = noticia.find('div', attrs={'class': 'bstn-related'})
    descricao = descricao.text if descricao else None # Somente acessará a propriedade text caso tenha encontrado ("find")
descricao

#Para extração da seção e do tempo decorrido, vamos acessar primeiro o atributo 'feed-post-metadata' e guardar em uma variável, 
# para em seguida, dentro desse novo subconjunto, localizar os atributos 'feed-post-datetime' e 'feed-post-metadata-section'. 
#Como existe a possibilidade dessa informação não existir, precisamos garantir que somente acessaremos a propriedade text  caso tenha encontrando ("find"). 
metadados = lista_noticias[0].find('div', attrs={'class':'feed-post-metadata'})

time_delta = metadados.find('span', attrs={'class': 'feed-post-datetime'})
secao = metadados.find('span', attrs={'class': 'feed-post-metadata-section'})

time_delta = time_delta.text if time_delta else None
secao = secao.text if secao else None
    
print('time_delta = ', time_delta)
print('seção = ', secao)

#cada extração deve ser feita dentro de uma estrutura de repetição
dados = []

for noticia in lista_noticias:
    manchete = noticia.contents[1].text.replace('"',"")
    link = noticia.find('a').get('href')

    descricao = noticia.contents[2].text
    if not descricao:
        descricao = noticia.find('div', attrs={'class': 'bstn-related'})
        descricao = descricao.text if descricao else None

    metadados = noticia.find('div', attrs={'class':'feed-post-metadata'})
    time_delta = metadados.find('span', attrs={'class': 'feed-post-datetime'})
    secao = metadados.find('span', attrs={'class': 'feed-post-metadata-section'})

    time_delta = time_delta.text if time_delta else None
    secao = secao.text if secao else None

    dados.append((manchete, descricao, link, secao, hora_extracao, time_delta))

df = pd.DataFrame(dados, columns=['manchete', 'descrição', 'link', 'seção', 'hora_extração', 'time_delta'])
df.head()




In [2]:
from datetime import datetime

import requests
from bs4 import BeautifulSoup
import pandas as pd

class ExtracaoPortal:
    def __init__(self):
        self.portal = None
    
    def extrair(self, portal):
        self.portal = portal
        texto_string = requests.get('https://globoesporte.globo.com/').text
        hora_extracao = datetime.now().strftime("%d-%m-%Y %H:%M:%S")

        bsp_texto = BeautifulSoup(texto_string, 'html.parser')
        lista_noticias = bsp_texto.find_all('div', attrs={'class':'feed-post-body'})
        
        dados = []

        for noticia in lista_noticias:
            manchete = noticia.contents[1].text.replace('"',"")
            link = noticia.find('a').get('href')

            descricao = noticia.contents[2].text
            if not descricao:
                descricao = noticia.find('div', attrs={'class': 'bstn-related'})
                descricao = descricao.text if descricao else None

            metadados = noticia.find('div', attrs={'class':'feed-post-metadata'})
            time_delta = metadados.find('span', attrs={'class': 'feed-post-datetime'})
            secao = metadados.find('span', attrs={'class': 'feed-post-metadata-section'})

            time_delta = time_delta.text if time_delta else None
            secao = secao.text if secao else None

            dados.append((manchete, descricao, link, secao, hora_extracao, time_delta))

        df = pd.DataFrame(dados, columns=['manchete', 'descrição', 'link', 'seção', 'hora_extração', 'time_delta'])
        return df

In [3]:
df = ExtracaoPortal().extrair("https://globoesporte.globo.com/")
df.head()

,manchete,descrição,link,seção,hora_extração,time_delta
0,"Com novo contrato no Catar, Xavi recebeu conta...","Em crise na CBF, Caboclo deu sua versão à comi...",https://globoesporte.globo.com/futebol/selecao...,seleção brasileira,19-05-2021 02:02:56,Há 1 hora
1,Corinthians apresenta projeto a agente de Rena...,Douglas diz que Pato quase foi agredido no Cor...,https://globoesporte.globo.com/futebol/times/c...,corinthians,19-05-2021 02:02:56,Há 3 horas
2,Willian lamenta gols bobos em primeira derrota...,Atacante pede agora foco na final do Paulistão...,https://globoesporte.globo.com/futebol/times/p...,palmeiras,19-05-2021 02:02:56,Há 1 hora
3,"Após derrota do Santos, Lucas Braga vê classif...",Auxiliar de Diniz minimiza falhas da defesa do...,https://globoesporte.globo.com/sp/santos-e-reg...,santos,19-05-2021 02:02:56,Há 56 minutos
4,Pacers eliminam os Hornets e vão disputar últi...,Primeiro jogo do play-in foi dominado pelo tim...,https://globoesporte.globo.com/basquete/nba/no...,nba,19-05-2021 02:02:56,Há 1 hora
